# 9. Improving retention with long short-term memory networks

## 9.1. LSTM

In [1]:
maxlen = 400
batch_size = 32
embedding_dims = 300
epochs = 2

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM

num_neurons = 50

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

2022-12-12 10:26:27.581805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 10:26:28.297698: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 10:26:28.297749: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-12 10:26:31.034169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

Build model...


2022-12-12 10:26:34.149046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-12 10:26:34.150405: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-12 10:26:34.150844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fe374cb43607): /proc/driver/nvidia/version does not exist
2022-12-12 10:26:34.153598: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 400, 50)           70200     
                                                                 
 dropout (Dropout)           (None, 400, 50)           0         
                                                                 
 flatten (Flatten)           (None, 20000)             0         
                                                                 
 dense (Dense)               (None, 1)                 20001     
                                                                 
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________
None


### 9.1.1 Backpropogration through time

In [2]:
import glob
import os
from random import shuffle

from nltk.tokenize import TreebankWordTokenizer
from gensim.models import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('../../bigdata/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=200000)

In [3]:
def pre_process_data(filepath):
    """
    This is dependent on your training data source but we will try to generalize it as best as possible.
    """
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')

    pos_label = 1
    neg_label = 0

    dataset = []

    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))

    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))

    shuffle(dataset)

    return dataset

In [4]:
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])

            except KeyError:
                pass  # No matching token in the Google w2v vocab

        vectorized_data.append(sample_vecs)

    return vectorized_data

In [5]:
def collect_expected(dataset):
    """ Peel of the target values from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

In [6]:
## http://ai.stanford.edu/~amaas/data/sentiment/
dataset = pre_process_data('../../bigdata/aclImdb/train/')

vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

split_point = int(len(vectorized_data) * .8)

In [7]:
maxlen = 400
batch_size = 32         # How many samples to show the net before backpropogating the error and updating the weights
embedding_dims = 300    # Length of the token vectors we will create for passing into the Convnet
# filters = 250           # Number of filters we will train
# kernel_size = 3         # The width of the filters, actual filters will each be a matrix of weights of size: embedding_dims x kernel_size or 50 x 3 in our case
# hidden_dims = 250       # Number of neurons in the plain feed forward net at the end of the chain
epochs = 2              # Number of times we will pass the entire training dataset through the network

In [8]:
def pad_trunc(data, maxlen):
    """ For a given dataset pad with zero vectors or truncate to maxlen """
    new_data = []

    # Create a vector of 0's the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:

        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [9]:
x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [10]:
## need to artificially reduce the size of test/train data for it to run on my machine
## then do the pad_trunc as documented

reducer = 8
print(f"Reduce test/train data by factor of {reducer}")

print(f"Before changes: {len(x_train)} and {len(x_test)}")

x_train = x_train[:int(len(x_train) / reducer)]
x_test = x_test[:int(len(x_test) / reducer)]

y_train = y_train[:int(len(y_train) / reducer)]
y_test = y_test[:int(len(y_test) / reducer)]

print(f"After reduction: {len(x_train)} and {len(x_test)}")

x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

print(f"After pad_trunc: {len(x_train)} and {len(x_test)}")
print(maxlen, embedding_dims)

Reduce test/train data by factor of 8
Before changes: 20000 and 5000
After reduction: 2500 and 625
After pad_trunc: 2500 and 625
400 300


In [11]:
import numpy as np

In [12]:
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM

num_neurons = 50

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

Build model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 400, 50)           70200     
                                                                 
 dropout_1 (Dropout)         (None, 400, 50)           0         
                                                                 
 flatten_1 (Flatten)         (None, 20000)             0         
                                                                 
 dense_1 (Dense)             (None, 1)                 20001     
                                                                 
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
model_structure = model.to_json()
with open("lstm_model1.json", "w") as json_file:
    json_file.write(model_structure)

model.save_weights("lstm_weights1.h5")
print('Model saved.')

2022-12-12 10:50:25.161830: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1200000000 exceeds 10% of free system memory.


Epoch 1/2
79/79 [==============================] - 64s 594ms/step - loss: 0.6306 - accuracy: 0.6492 - val_loss: 0.4786 - val_accuracy: 0.7712
Epoch 2/2
79/79 [==============================] - 32s 407ms/step - loss: 0.4948 - accuracy: 0.7724 - val_loss: 0.4391 - val_accuracy: 0.8000
Model saved.


### 9.1.2 Where does the rubber hit the road?

In [15]:
from keras.models import model_from_json
with open("lstm_model1.json", "r") as json_file:
    json_string = json_file.read()
model = model_from_json(json_string)

model.load_weights('lstm_weights1.h5')

In [17]:
sample_1 = "I hate that the dismal weather that had me down for so long, when will it break! Ugh, when does happiness return?  The sun is blinding and the puffy clouds are too thin.  I can't wait for the weekend."

# We pass a dummy value in the first element of the tuple just because our helper expects it from the way processed the initial data.  That value won't ever see the network, so it can be whatever.
vec_list = tokenize_and_vectorize([(1, sample_1)])

# Tokenize returns a list of the data (length 1 here)
test_vec_list = pad_trunc(vec_list, maxlen)

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# print("Sample's sentiment, 1 - pos, 2 - neg : {}".format(model.predict_classes(test_vec)))
# print("Raw output of sigmoid function: {}".format(model.predict(test_vec)))
## AttributeError: 'Sequential' object has no attribute 'predict_classes'

predict_x = model.predict(test_vec) 
classes_x = np.argmax(predict_x,axis=1)

print("Sample's sentiment, 1 - pos, 0 - neg : {}".format(classes_x))
print("Raw output of sigmoid function: {}".format(predict_x))

1/1 [==============================] - 1s 861ms/step
Sample's sentiment, 1 - pos, 2 - neg : [0]
Raw output of sigmoid function: [[0.39549184]]


### 9.1. Dirty data

In [22]:
# !tar -xvzf aclImdb_v1.tar.gz

In [24]:
def test_len(data, maxlen):
    total_len = truncated = exact = padded = 0
    for sample in data:
        total_len += len(sample)
        if len(sample) > maxlen:
            truncated += 1
        elif len(sample) < maxlen:
            padded += 1
        else:
            exact +=1 
    print('Padded: {}'.format(padded))
    print('Equal: {}'.format(exact))
    print('Truncated: {}'.format(truncated))
    print('Avg length: {}'.format(total_len/len(data)))

# dataset = pre_process_data('./aclImdb_v1/train')
dataset = pre_process_data('../../bigdata/aclImdb/train/')
vectorized_data = tokenize_and_vectorize(dataset)
test_len(vectorized_data, 400)

Padded: 22560
Equal: 12
Truncated: 2428
Avg length: 202.43216


In [ ]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM


maxlen = 200
batch_size = 32         # How many samples to show the net before backpropagating the error and updating the weights
embedding_dims = 300    # Length of the token vectors we will create for passing into the Convnet

epochs = 2

# dataset = pre_process_data('./aclImdb_v1/train')
dataset = pre_process_data('../../bigdata/aclImdb/train/')
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

split_point = int(len(vectorized_data)*.8)

In [28]:
x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

## need to artificially reduce the size of test/train data for it to run on my machine
## then do the pad_trunc as documented

reducer = 8
print(f"Reduce test/train data by factor of {reducer}")

print(f"Before changes: {len(x_train)} and {len(x_test)}")

x_train = x_train[:int(len(x_train) / reducer)]
x_test = x_test[:int(len(x_test) / reducer)]

y_train = y_train[:int(len(y_train) / reducer)]
y_test = y_test[:int(len(y_test) / reducer)]

print(f"After reduction: {len(x_train)} and {len(x_test)}")

x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

print(f"After pad_trunc: {len(x_train)} and {len(x_test)}")
print(maxlen, embedding_dims)

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

num_neurons = 50

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

Reduce test/train data by factor of 8
Before changes: 20000 and 5000
After reduction: 2500 and 625
After pad_trunc: 2500 and 625
200 300
Build model...
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 200, 50)           70200     
                                                                 
 dropout_3 (Dropout)         (None, 200, 50)           0         
                                                                 
 flatten_3 (Flatten)         (None, 10000)             0         
                                                                 
 dense_3 (Dense)             (None, 1)                 10001     
                                                                 
Total params: 80,201
Trainable params: 80,201
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
model_structure = model.to_json()
with open("lstm_model7.json", "w") as json_file:
    json_file.write(model_structure)

model.save_weights("lstm_weights7.h5")
print('Model saved.')

2022-12-12 14:47:11.622780: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 600000000 exceeds 10% of free system memory.


Epoch 1/2
79/79 [==============================] - 43s 394ms/step - loss: 0.6436 - accuracy: 0.6080 - val_loss: 0.5483 - val_accuracy: 0.7440
Epoch 2/2
79/79 [==============================] - 19s 245ms/step - loss: 0.5106 - accuracy: 0.7616 - val_loss: 0.9454 - val_accuracy: 0.5920
Model saved.


### 9.1.5 Words are hard. Letters are easier.

In [30]:
# dataset = pre_process_data('./aclImdb_v1/train')
# dataset = pre_process_data('../../bigdata/aclImdb_v1/train/')
# expected = collect_expected(dataset)

In [31]:
def avg_len(data):
    total_len = 0
    for sample in data:
        total_len += len(sample[1])
    print(total_len/len(data))

print(avg_len(dataset))

1325.06964
None


In [32]:
def clean_data(data):
    """ Shift to lower case, replace unknowns with UNK, and listify """
    new_data = []
    VALID = 'abcdefghijklmnopqrstuvwxyz123456789"\'?!.,:; '
    for sample in data:
        new_sample = []
        for char in sample[1].lower():  # Just grab the string, not the label
            if char in VALID:
                new_sample.append(char)
            else:
                new_sample.append('UNK')
       
        new_data.append(new_sample)
    return new_data

listified_data = clean_data(dataset)

In [33]:
def char_pad_trunc(data, maxlen):
    """ We truncate to maxlen or add in PAD tokens """
    new_dataset = []
    for sample in data:
        if len(sample) > maxlen:
            new_data = sample[:maxlen]
        elif len(sample) < maxlen:
            pads = maxlen - len(sample)
            new_data = sample + ['PAD'] * pads
        else:
            new_data = sample
        new_dataset.append(new_data)
    return new_dataset

maxlen = 1500

In [34]:
def create_dicts(data):
    """ Modified from Keras LSTM example"""
    chars = set()
    for sample in data:
        chars.update(set(sample))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    return char_indices, indices_char

In [35]:
import numpy as np

def onehot_encode(dataset, char_indices, maxlen):
    """ 
    One hot encode the tokens
    
    Args:
        dataset  list of lists of tokens
        char_indices  dictionary of {key=character, value=index to use encoding vector}
        maxlen  int  Length of each sample
    Return:
        np array of shape (samples, tokens, encoding length)
    """
    X = np.zeros((len(dataset), maxlen, len(char_indices.keys())))
    for i, sentence in enumerate(dataset):
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
    return X

In [37]:
# dataset = pre_process_data('./aclImdb_v1/train')
dataset = pre_process_data('../../bigdata/aclImdb_v1/train/')
expected = collect_expected(dataset)
listified_data = clean_data(dataset)

maxlen = 1500
common_length_data = char_pad_trunc(listified_data, maxlen)

char_indices, indices_char = create_dicts(common_length_data)
encoded_data = onehot_encode(common_length_data, char_indices, maxlen)

In [38]:
split_point = int(len(encoded_data)*.8)

x_train = encoded_data[:split_point]
y_train = expected[:split_point]
x_test = encoded_data[split_point:]
y_test = expected[split_point:]

In [41]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten, LSTM


num_neurons = 40

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, len(char_indices.keys()))))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'], run_eagerly=True)
print(model.summary())

Build model...
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 1500, 40)          6560      
                                                                 
 dropout_5 (Dropout)         (None, 1500, 40)          0         
                                                                 
 flatten_5 (Flatten)         (None, 60000)             0         
                                                                 
 dense_5 (Dense)             (None, 1)                 60001     
                                                                 
Total params: 66,561
Trainable params: 66,561
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
batch_size = 32
epochs = 10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
model_structure = model.to_json()
with open("char_lstm_model3.json", "w") as json_file:
    json_file.write(model_structure)

model.save_weights("char_lstm_weights3.h5")
print('Model saved.')

Epoch 1/10


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

### 9.1.7 My turn to speak more clearly

In [46]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [47]:
from nltk.corpus import gutenberg

print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [48]:
text = ''
for txt in gutenberg.fileids():
    if 'shakespeare' in txt:
        text += gutenberg.raw(txt).lower()

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 375542
total chars: 50


In [49]:
print(text[:500])

[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a carpenter

   mur. where is thy leather apron, and thy rule?
what dost thou with thy best apparrell on


In [50]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 125168


In [51]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


/tmp/ipykernel_1904/3464639879.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
/tmp/ipykernel_1904/3464639879.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [52]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

print(model.summary())

Build model...
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 128)               91648     
                                                                 
 dense_6 (Dense)             (None, 50)                6450      
                                                                 
 activation (Activation)     (None, 50)                0         
                                                                 
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________
None


/opt/conda/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [53]:
epochs = 6
batch_size = 128

model_structure = model.to_json()
with open("shakes_lstm_model.json", "w") as json_file:
    json_file.write(model_structure)

for i in range(5):
    model.fit(X, y,
              batch_size=batch_size,
              epochs=epochs)

    model.save_weights("shakes_lstm_weights_{}.h5".format(i+1))
    print('Model saved.')

Epoch 1/6
978/978 [==============================] - 93s 91ms/step - loss: 2.0414
Epoch 2/6
978/978 [==============================] - 92s 94ms/step - loss: 1.6827
Epoch 3/6
978/978 [==============================] - 99s 101ms/step - loss: 1.5783
Epoch 4/6
978/978 [==============================] - 91s 93ms/step - loss: 1.5172
Epoch 5/6
978/978 [==============================] - 79s 81ms/step - loss: 1.4740
Epoch 6/6
978/978 [==============================] - 83s 85ms/step - loss: 1.4429
Model saved.
Epoch 1/6
978/978 [==============================] - 80s 82ms/step - loss: 1.4172
Epoch 2/6
978/978 [==============================] - 101s 104ms/step - loss: 1.3980
Epoch 3/6
978/978 [==============================] - 105s 107ms/step - loss: 1.3795
Epoch 4/6
978/978 [==============================] - 97s 99ms/step - loss: 1.3641
Epoch 5/6
978/978 [==============================] - 89s 91ms/step - loss: 1.3532
Epoch 6/6
978/978 [==============================] - 90s 92ms/step - loss: 1.342

In [54]:
### NOT IN CHAPTER, Just to reproduce output

from keras.models import model_from_json

with open('shakes_lstm_model.json', 'r') as f:
    model_json = f.read()
    
model = model_from_json(model_json)
model.load_weights('shakes_lstm_weights_4.h5')

In [55]:
import random

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [56]:
import sys

start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "he.
(o heauen! a beast that wants discou"
he.
(o heauen! a beast that wants discoue thee:
but what is the stand of the straight,
that may be made me to the second byrnanes

   cassi. i haue seene thee thinke the tran to the strikes,
and the straine, and the connerous to the secrees

   cassi. shall we must you this point, and the state
in the chan and be a right and so much vnrtyes the most conity,
the chances of the straight, and the secree

   cassi. so the secret and the rea

----- diversity: 0.5
----- Generating with seed: "he.
(o heauen! a beast that wants discou"
he.
(o heauen! a beast that wants discouer, state
the world be receiue a constanc'd display,
who haue i you shall he is a man charge,
i should welo not be a cassius, when you know not

   ross. thou louicus you do you are you,
and with the straiges and haue made me forme,
i loue me the second king, which it selfe in fire the side,
become my sight, and all the selues bear

### 9.1.9 Other kinds of memory

In [57]:
from keras.models import Sequential
from keras.layers import GRU

model = Sequential()
model.add(GRU(num_neurons, return_sequences=True, input_shape=X[0].shape))

### 9.1.10 Going deeper

In [59]:
num_neurons_2 = num_neurons

In [60]:
from keras.models import Sequential
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(num_neurons, return_sequences=True, input_shape=X[0].shape))
model.add(LSTM(num_neurons_2, return_sequences=True))